In [ ]:
from PIL import Image, ImageFilter, ImageOps
import matplotlib.pyplot as plt
import pyocr
import numpy as np
import cv2
import glob

def cv2pil(image):
    new_image = image.copy()
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGRA2RGBA)
    new_image = Image.fromarray(new_image)
    return new_image
def pil2cv(image):
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2:  # モノクロ
        pass
    elif new_image.shape[2] == 3:  # カラー
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:  # 透過
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

def is_num(s):
    try:
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import pyocr
import numpy as np
import cv2
import glob

img = cv2.imread('logger/frame_0002.jpg')

img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_RGB)


In [ ]:
plt.gray()
im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

plt.imshow(im_gray)

In [ ]:
# [y:y+h, x:x+w]
# img_tr = im_gray[300:800, 5:740]
img_tr1 = im_gray[20:140, 110:400]
img_tr2 = im_gray[20:140, 500:800]
# img_tr3 = im_gray[106:138, 400:500]
# img_tr4 = im_gray[172:200, 400:500]

plt.imshow(img_tr1)

# cv2.imwrite('tirm.jpg', img_tr)

In [ ]:
ret1, dst1 = cv2.threshold(img_tr1, 128, 255, cv2.THRESH_OTSU)
ret2, dst2 = cv2.threshold(img_tr2, 128, 255, cv2.THRESH_OTSU)
# ret3, dst3 = cv2.threshold(img_tr3, 128, 255, cv2.THRESH_OTSU)
# ret4, dst4 = cv2.threshold(img_tr4, 128, 255, cv2.THRESH_OTSU)
plt.imshow(dst1)
print(ret1)
#   th, im_otsu = cv2.threshold(pil2cv(gray_img), 128, 192, cv2.THRESH_OTSU)

In [ ]:
# 色の反転
img_thresh1 = cv2.bitwise_not(img_tr1)
img_thresh2 = cv2.bitwise_not(dst2)
# img_thresh3 = cv2.bitwise_not(dst3)
# img_thresh4 = cv2.bitwise_not(dst4)

plt.imshow(img_thresh1)
# cv2.imwrite('trin_反転.jpg', img_thresh)

In [ ]:

tools = pyocr.get_available_tools()
tool = tools[0]

pil_image = Image.fromarray(img_thresh1)

txt = tool.image_to_string(
  pil_image,
  lang='jpn',
  # lang='eng',
  builder=pyocr.builders.DigitBuilder(tesseract_layout=6)
)
print(txt)

In [ ]:
'''
画像処理+文字認識
'''
cl_lst = []
file_lst = []
engines = pyocr.get_available_tools()
engine = engines[0]
files = glob.glob("yoshimi/*")
files.sort()
for file in files:
    '''
    ===========画像処理開始==========
    '''
    #グレースケール
    img = cv2.imread(file)
    # img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.gray()
    im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img_tr = im_gray[620:660, 650:735]
    img_tr1 = im_gray[42:72, 450:500]
    img_tr2 = im_gray[73:105, 400:500]
    img_tr3 = im_gray[106:138, 400:500]
    img_tr4 = im_gray[172:200, 400:500]


    ret1, dst1 = cv2.threshold(img_tr1, 128, 255, cv2.THRESH_OTSU)
    ret2, dst2 = cv2.threshold(img_tr2, 128, 255, cv2.THRESH_OTSU)
    ret3, dst3 = cv2.threshold(img_tr3, 128, 255, cv2.THRESH_OTSU)
    ret4, dst4 = cv2.threshold(img_tr4, 128, 255, cv2.THRESH_OTSU)
    img_thresh1 = cv2.bitwise_not(dst1)
    img_thresh2 = cv2.bitwise_not(dst2)
    img_thresh3 = cv2.bitwise_not(dst3)
    img_thresh4 = cv2.bitwise_not(dst4)
    
    
    a = file[1:]
    # print(a)
    cv2.imwrite('result/' + a, img_thresh4)
    pil_image = Image.fromarray(img_thresh4)
    #文字認識
    txt = engine.image_to_string(pil_image, lang="eng",
                                 builder=pyocr.builders.TextBuilder(tesseract_layout=6))
    # at = txt.find("mg")
    print("==========", file, "==========")
    print(txt)
    
    if not txt:
        cl_lst.append(-100)
    else:
        cl_lst.append(txt)
    
    file_lst.append(file)
    


In [ ]:
import pandas as pd
cl = pd.DataFrame(cl_lst)
f_lst = pd.DataFrame(file_lst)

In [ ]:
df = pd.concat([f_lst, cl], axis=1)
df

In [ ]:
df.to_csv('data.csv')